# Exercicio 7 - DELETE
**AiDAPT - Cegid Academy**

Conceitos: DELETE, subconsultas, NOT IN, NOT EXISTS

Base de dados: ExerciciosDB

> **ATENCAO**: Estas queries ELIMINAM dados! Recomenda-se fazer backup ou usar transaccoes.

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from urllib.parse import quote_plus
from sqlalchemy import create_engine
load_dotenv(find_dotenv())

%load_ext sql

host = os.getenv('MSSQL_HOST', 'localhost')
port = os.getenv('MSSQL_PORT', '1433')
user = os.getenv('MSSQL_USER', 'sa')
password = quote_plus(os.getenv('MSSQL_PASSWORD', 'your_password_here'))
engine = create_engine(f"mssql+pymssql://{user}:{password}@{host}:{port}/ExerciciosDB")
%sql engine --alias ExerciciosDB

> **NOTA**: O JupySQL nao suporta transaccoes (`BEGIN TRANSACTION` / `ROLLBACK`).
> Por isso, os DELETEs usam SQLAlchemy directamente (`engine.connect()`) com `rollback()`,
> pelo que **nenhum dado e efectivamente eliminado**.
> As queries de verificacao (ANTES/DEPOIS) continuam em `%%sql` puro.

## 7.1 Remover pagamentos FAILED

### Verificar ANTES

In [ ]:
%%sql
SELECT *
FROM dbo.Payments
WHERE Status = 'FAILED';

### Executar DELETE

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    trans = conn.begin()
    result = conn.execute(text("""
        DELETE FROM dbo.Payments
        WHERE Status = 'FAILED'
    """))
    print(f"Linhas eliminadas: {result.rowcount}")
    trans.rollback()
    print("ROLLBACK executado — dados preservados")

### Verificar DEPOIS

In [ ]:
%%sql
SELECT COUNT(*) AS PagamentosFailed
FROM dbo.Payments
WHERE Status = 'FAILED';

---
## 7.2 Remover pedidos CANCELLED (se nao houver pagamentos confirmados)

### Verificar ANTES

In [ ]:
%%sql
SELECT o.OrderID, o.Status, p.PaymentID, p.Status AS PaymentStatus
FROM dbo.Orders o
LEFT JOIN dbo.Payments p ON o.OrderID = p.OrderID
WHERE o.Status = 'CANCELLED';

### Executar DELETE

Para eliminar Orders, e necessario eliminar primeiro os OrderItems (tabela filha).
A FK `FK_OrderItems_Orders` com `ON DELETE NO_ACTION` impede eliminar Orders que tenham itens.

Dentro da transaccao: eliminar filhos (OrderItems) → depois pais (Orders) → ROLLBACK.

> **Alternativa sem transaccao (opcao 1)**: Eliminar OrderItems e Orders definitivamente,
> sem ROLLBACK. Nesse caso os dados seriam efectivamente removidos. A ordem seria a mesma
> (filhos antes de pais), mas com `COMMIT` ou sem transaccao explicita.

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    trans = conn.begin()

    # 1. Eliminar OrderItems dos pedidos CANCELLED (filhos antes de pais)
    r1 = conn.execute(text("""
        DELETE FROM dbo.OrderItems
        WHERE OrderID IN (
            SELECT OrderID FROM dbo.Orders
            WHERE Status = 'CANCELLED'
              AND OrderID NOT IN (
                  SELECT OrderID FROM dbo.Payments WHERE Status = 'CONFIRMED'
              )
        )
    """))
    print(f"OrderItems eliminados: {r1.rowcount}")

    # 2. Eliminar os pedidos CANCELLED
    r2 = conn.execute(text("""
        DELETE FROM dbo.Orders
        WHERE Status = 'CANCELLED'
          AND OrderID NOT IN (
              SELECT OrderID
              FROM dbo.Payments
              WHERE Status = 'CONFIRMED'
          )
    """))
    print(f"Orders eliminados: {r2.rowcount}")

    trans.rollback()
    print("ROLLBACK executado — dados preservados")

---
## 7.3 Remover produtos inativos que nunca foram vendidos

### Verificar ANTES

In [ ]:
%%sql
SELECT p.ProductID, p.ProductName, p.Active
FROM dbo.Products p
WHERE p.Active = 0
  AND p.ProductID NOT IN (
      SELECT DISTINCT ProductID
      FROM dbo.OrderItems
  );

### Executar DELETE

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    trans = conn.begin()
    result = conn.execute(text("""
        DELETE FROM dbo.Products
        WHERE Active = 0
          AND ProductID NOT IN (
              SELECT DISTINCT ProductID
              FROM dbo.OrderItems
          )
    """))
    print(f"Linhas eliminadas: {result.rowcount}")
    trans.rollback()
    print("ROLLBACK executado — dados preservados")

---
## 7.4 Remover clientes sem pedido

### Verificar ANTES

In [ ]:
%%sql
SELECT c.CustomerID, c.FullName
FROM dbo.Customers c
WHERE c.CustomerID NOT IN (
    SELECT DISTINCT CustomerID
    FROM dbo.Orders
);

### Executar DELETE

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    trans = conn.begin()
    result = conn.execute(text("""
        DELETE FROM dbo.Customers
        WHERE CustomerID NOT IN (
            SELECT DISTINCT CustomerID
            FROM dbo.Orders
        )
    """))
    print(f"Linhas eliminadas: {result.rowcount}")
    trans.rollback()
    print("ROLLBACK executado — dados preservados")

## Notas

- **SEMPRE** usar WHERE no DELETE (senao elimina TUDO!)
- Verificar dados ANTES de eliminar
- Considerar FOREIGN KEYS: eliminar filhos antes de pais
- `NOT IN` vs `NOT EXISTS`: NOT EXISTS e geralmente mais eficiente

### Transaccoes neste notebook

Todos os DELETEs estao envolvidos em `BEGIN TRANSACTION` / `ROLLBACK`:
- O DELETE executa normalmente (mostra "N rows affected")
- O ROLLBACK desfaz a operacao — os dados permanecem intactos
- "Verificar DEPOIS" confirma que os dados nao foram alterados

### Alternativa: eliminar dados definitivamente (sem ROLLBACK)

Para eliminar dados de facto, a abordagem correcta seria:
1. Eliminar registos das **tabelas filhas** primeiro (OrderItems, Payments)
2. Depois eliminar registos das **tabelas pai** (Orders, Customers)
3. Nao usar ROLLBACK (ou usar `COMMIT`)

Esta ordem e obrigatoria quando as FK usam `ON DELETE NO_ACTION` (comportamento por defeito).
Com `ON DELETE CASCADE`, o SQL Server eliminaria automaticamente os filhos.